<a href="https://colab.research.google.com/github/cicattzo/nlp_project/blob/main/6_684_group_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
"""
Install working versions of packages
"""
!pip install torch==1.4.1
!pip install datasets==1.0.2
# !pip install transformers==4.0.1
!pip install transformers==4.4.2


In [ ]:
import datasets
# import datasets
train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602. Subsequent calls will reuse this data.


In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [ ]:
train_data[0]['highlights']

'Syrian official: Obama climbed to the top of the tree, "doesn\'t know how to get down"\nObama sends a letter to the heads of the House and Senate .\nObama to seek congressional approval on military action against Syria .\nAim is to determine whether CW were used, not by whom, says U.N. spokesman .'

In [ ]:
encoder_max_length=512
decoder_max_length=128

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

In [ ]:
batch_size=4

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights", "id"]
)

In [ ]:
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [ ]:
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)


In [ ]:
val_data = val_data.select(range(8))

In [ ]:
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights", "id"]
)

In [ ]:
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

## BC3 Corpus Pre-processing

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%bash
pip install talon

cd "/content/gdrive/MyDrive/6864_project/"

  Created wheel for talon: filename=talon-1.4.4-cp37-none-any.whl size=35288 sha256=c4f6fba9b1972b57a87e9cf30d679971e6f6d7014ae50eb72b567c5647fff410
  Stored in directory: /root/.cache/pip/wheels/3f/ca/f9/748f5328839c03fe055f69110357b4718853dde8d872896209
Successfully built talon


In [ ]:
import sys
import pandas as pd
import numpy as np
import re
import xml.etree.ElementTree as ET
from talon.signature.bruteforce import extract_signature
import email

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# don't need this function
# def process_date(date_time):
#     '''
#     Converts the MIME date format to a more pandas friendly type. 
#     '''
#     try:
#         date_time = email.utils.format_datetime(email.utils.parsedate_to_datetime(date_time))
#     except:
#         date_time = None
#     return date_time


def clean_body(mail_body):
    '''
    This extracts both the email signature, and the forwarding email chain if it exists. 
    '''
    delimiters = ["-----Original Message-----","To:","From"]
    
    #Trying to split string by biggest delimiter. 
    old_len = sys.maxsize
    
    for delimiter in delimiters:
        split_body = mail_body.split(delimiter,1)
        new_len = len(split_body[0])
        if new_len <= old_len:
            old_len = new_len
            final_split = split_body
            
    #Then pull chain message
    if (len(final_split) == 1):
        mail_chain = None
    else:
        mail_chain = final_split[1] 
    
    # The signature can be removed from the body using Talon
    clean_body, sig = extract_signature(final_split[0]) 
    # clean_body = final_split[0]
    
    return {
            'body': clean_body, 
            'chain' : mail_chain, 
            # 'signature': sig
            }

In [ ]:
def parse_bc3_emails(root):
    '''
    This adds every BC3 email to a newly created dataframe. 
    '''
    BC3_email_list = []
    #The emails are seperated by threads.
    for thread in root:
        email_num = 0
        #Iterate through the thread elements <name, listno, Doc>
        for thread_element in thread:
            #Getting the listno allows us to link the summaries to the correct emails
            if thread_element.tag == "listno":
                listno = thread_element.text
            #Each Doc element is a single email
            if thread_element.tag == "DOC":
                email_num += 1
                email_metadata = []
                for email_attribute in thread_element:
                    #If the email_attri is text, then each child contains a line from the body of the email
                    if email_attribute.tag == "Text":
                        email_body = ""
                        for sentence in email_attribute:
                            email_body += sentence.text
                    else:
                        #The attributes of the Email <Recieved, From, To, Subject, Text> appends in this order. 
                        email_metadata.append(email_attribute.text)
                        
                #Use same enron cleaning methods on the body of the email
                split_body = clean_body(email_body)
                    
                email_dict = {
                    "listno" : listno,
                    # "date" : process_date(email_metadata[0]),  # don't need date, to and from
                    # "from" : email_metadata[1],
                    # "to" : email_metadata[2],
                    "subject" : email_metadata[3],
                    "body" : split_body['body'],
                    "email_num": email_num
                }
                
                BC3_email_list.append(email_dict)           
    return pd.DataFrame(BC3_email_list)

In [ ]:
def parse_bc3_summaries(root):
    '''
    This parses every BC3 Human summary that is contained in the dataset. 
    '''
    BC3_summary_list = []
    for thread in root:
        #Iterate through the thread elements <listno, name, annotation>
        for thread_element in thread:
            if thread_element.tag == "listno":
                listno = thread_element.text
            #Each Doc element is a single email
            if thread_element.tag == "annotation":
                for annotation in thread_element:
                #If the email_attri is summary, then each child contains a summarization line
                    if annotation.tag == "summary":
                        summary_dict = {}
                        for summary in annotation:
                            #Generate the set of emails the summary sentence belongs to (often a single email)
                            email_nums = summary.attrib['link'].split(',')
                            s = set()
                            for num in email_nums:
                                s.add(num.split('.')[0].strip()) 
                            #Remove empty strings, since they summarize whole threads instead of emails. 
                            s = [x for x in set(s) if x]
                            for email_num in s:
                                if email_num in summary_dict:
                                    summary_dict[email_num] += ' ' + summary.text
                                else:
                                    summary_dict[email_num] = summary.text
                    #get annotator description
                    # elif annotation.tag == "desc":
                    #     annotator = annotation.text
                #For each email summarizaiton create an entry
                for email_num, summary in summary_dict.items():
                    email_dict = {
                        "listno" : listno,
                        # "annotator" : annotator,
                        "email_num" : email_num,
                        "summary" : summary
                    }      
                    BC3_summary_list.append(email_dict)
    return pd.DataFrame(BC3_summary_list)

In [ ]:
load_saved = True

if not load_saved:
  #load BC3 Email Corpus
  parsedXML = ET.parse( "/content/gdrive/My Drive/6864_project/bc3.1.0/corpus.xml" )
  root = parsedXML.getroot()

  #Clean up BC3 emails the same way as the Enron emails. 
  bc3_email_df = parse_bc3_emails(root)

  #Load summaries and process
  parsedXML = ET.parse( "/content/gdrive/My Drive/6864_project/bc3.1.0/annotation.xml" )
  root = parsedXML.getroot()
  bc3_summary_df = parse_bc3_summaries(root)
  bc3_summary_df['email_num'] = bc3_summary_df['email_num'].astype(int)

  #merge the dataframes together
  bc3_df = pd.merge(bc3_email_df, 
                    bc3_summary_df[['email_num', 'listno', 'summary']],
                  on=['email_num', 'listno'])

  bc3_df.to_csv("/content/gdrive/My Drive/6864_project/bc3_processed.csv", index=False)
else:
  bc3_df = pd.read_csv("/content/gdrive/My Drive/6864_project/bc3_processed.csv")

In [ ]:
bc3_df.shape

(669, 5)

In [ ]:
bc3_df.head()

,listno,subject,body,email_num,summary
0,007-7484738,Extending IETF meetings to two weeks?,"The IETF meetings tend to become too large, cr...",1,"Jacob suggested to hold two week meetings, the..."
1,007-7484738,Extending IETF meetings to two weeks?,"The IETF meetings tend to become too large, cr...",1,Jacob suggests that future IETF meetings be sp...
2,007-7484738,Extending IETF meetings to two weeks?,"The IETF meetings tend to become too large, cr...",1,The topic is the logistics of scheduling IETF ...
3,007-7484738,Re: Extending IETF meetings to two weeks?,"> The IETF meetings tend to become too large, ...",2,Terry supported Jacob's idea and suggested a f...
4,007-7484738,Re: Extending IETF meetings to two weeks?,"> The IETF meetings tend to become too large, ...",2,Some argue that it'd be more useful to prepare...


In [ ]:
# keeping the subject and body separate, but they can be merged
bc3_df['unique_key'] = bc3_df['listno'] + "-" + bc3_df['email_num'].astype(str)
bc3_dict = bc3_df.groupby('unique_key').agg({'subject':lambda x: x.iloc[0], 'body':lambda x: x.iloc[0], 'summary':lambda x: x.to_list()}).to_dict()

In [ ]:
len(bc3_dict['body'])

259

Splitting data into test and train and saving

In [ ]:
N = bc3_df.shape[0]
ntrain = int(N*0.7)

bc3_train = bc3_df.iloc[:ntrain]
bc3_test = bc3_df.iloc[ntrain:]

In [ ]:
bc3_train.to_csv("/content/gdrive/My Drive/6864_project/bc3_train.csv", index=False)
bc3_test.to_csv("/content/gdrive/My Drive/6864_project/bc3_test.csv", index=False)

### Load and combine BC3 and CNN training data

In [ ]:
cnn_train = pd.read_csv("cnn_train_data_5.csv")

In [ ]:
cnn_train.drop(columns=["Unnamed: 0"], inplace=True)
cnn_train.head()

,article,highlights,id
0,It's official: U.S. President Barack Obama wan...,Syrian official: Obama climbed to the top of t...,0001d1afc246a7964130f43ae940af6bc6c57f01
1,(CNN) -- Usain Bolt rounded off the world cham...,Usain Bolt wins third gold of world championsh...,0002095e55fcbd3a2f366d9bf92a95433dc305ef
2,"Kansas City, Missouri (CNN) -- The General Ser...",The employee in agency's Kansas City office is...,00027e965c8264c35cc1bc55556db388da82b07f
3,Los Angeles (CNN) -- A medical doctor in Vanco...,NEW: A Canadian doctor says she was part of a ...,0002c17436637c4fe1837c935c04de47adb18e9a
4,(CNN) -- Police arrested another teen Thursday...,Another arrest made in gang rape outside Calif...,0003ad6ef0c37534f80b55b4235108024b407f0b


In [ ]:
bc3_train["article"] = bc3_train.subject + " \n" + bc3_train.body
bc3_train = bc3_train.drop(columns=['subject', 'body', 'email_num']).rename(columns={'listno':'id', 'summary':'highlights'})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
train_combined = pd.concat([cnn_train, bc3_train], axis=0)

In [ ]:
train_combined.to_csv("/content/gdrive/My Drive/6864_project/train_combined.csv", index=False)

## Converting to arrow dataset

In [ ]:
bc3_dataset = datasets.Dataset.from_pandas(bc3_df)

In [ ]:
bc3_dataset

Dataset(features: {'listno': Value(dtype='string', id=None), 'subject': Value(dtype='string', id=None), 'body': Value(dtype='string', id=None), 'email_num': Value(dtype='int64', id=None), 'summary': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None)}, num_rows: 669)

## Check Everything

In [ ]:
import pandas as pd

In [ ]:
train_combined = pd.read_csv("train_combined.csv")
train_combined.head()

,article,highlights,id
0,It's official: U.S. President Barack Obama wan...,Syrian official: Obama climbed to the top of t...,0001d1afc246a7964130f43ae940af6bc6c57f01
1,(CNN) -- Usain Bolt rounded off the world cham...,Usain Bolt wins third gold of world championsh...,0002095e55fcbd3a2f366d9bf92a95433dc305ef
2,"Kansas City, Missouri (CNN) -- The General Ser...",The employee in agency's Kansas City office is...,00027e965c8264c35cc1bc55556db388da82b07f
3,Los Angeles (CNN) -- A medical doctor in Vanco...,NEW: A Canadian doctor says she was part of a ...,0002c17436637c4fe1837c935c04de47adb18e9a
4,(CNN) -- Police arrested another teen Thursday...,Another arrest made in gang rape outside Calif...,0003ad6ef0c37534f80b55b4235108024b407f0b


In [ ]:
train_combined.iloc[11861]

article       (CNN) -- Real Madrid could be without star str...
highlights    Cristiano Ronaldo ruled out for the next three...
id                     21a5badc573527f822cfbd3f8dc1de0fab40e73a
Name: 11861, dtype: object